## Combining Multiple DGI Datasets:

This gives brief idea on how datasets can be combined for analysis/visualisation purposes. Three datasest are taken for the purposes of the demo. They are - 

<li>South Dublin Libraries</li>
<li>Limerick Libraries</li>
<li>Wexford Libraries</li>

##### Install requirements

In [14]:
!pip install numpy requests geopy googlemaps matplotlib seaborn pyjstat pandas fuzzywuzzy google folium

twisted 18.7.0 requires PyHamcrest>=1.9.0, which is not installed.
You are using pip version 10.0.1, however version 19.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


##### Import all necessary libraries:

In [15]:
import requests
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import sys
import pandas as pd
from pyjstat import pyjstat
import numpy as np
from pyjstat import pyjstat
import plotly.plotly as py
import plotly.graph_objs as go
import matplotlib.pyplot as plt
import seaborn as sns
import json
from geopy.geocoders import Nominatim
from googlemaps import googlemaps
from geopy.geocoders import GoogleV3
import time
import sys
from bs4 import BeautifulSoup
from googlesearch import search 
import urllib
import folium
import pandas

## Search all datasets with a given query:

This function returns the meta information about matching packages. Required resources can be selected from the result of the funtion.

In [16]:
def package_search(url, query):
    
    try:
        
        response = requests.get(url+'api/3/action/package_search', {'q':query})
        results = response.json()
        
        if results['success'] is not True:
            raise SystemError
            
        if len(results['result']) == 0:
            raise TypeError
            
        return results['result']

    except SystemError:
        
        print("API failure - ")
        sys.exit(1)
        
    except TypeError:
        
        print("No results found for supplied query!")
        sys.exit(1)
        
    except Exception as e:
        print(e)
        sys.exit(1)
    

res = package_search('https://data.gov.ie/', 'libraries')
print(res)

{'count': 29, 'sort': 'score desc, metadata_modified desc', 'facets': {}, 'results': [{'license_title': 'Creative Commons Attribution 4.0', 'maintainer': 'Not supplied', 'issued': '2013-12-18', 'private': False, 'maintainer_email': 'data@smartdublin.ie', 'num_tags': 3, 'frequency': 'Irregular', 'id': 'cd8ef111-10da-4abd-a9dc-0e82c9fd256e', 'metadata_created': '2015-09-13T15:34:06.081426', 'metadata_modified': '2018-03-05T15:54:11.594082', 'author': 'Not supplied', 'author_email': 'data@smartdublin.ie', 'temporal': '2013-01-01 to 2013-12-31', 'theme': 'Arts', 'state': 'active', 'version': '1.0', 'relationships_as_object': [], 'license_id': 'CC-BY-4.0', 'type': 'dataset', 'resources': [{'cache_last_updated': None, 'package_id': 'cd8ef111-10da-4abd-a9dc-0e82c9fd256e', 'datastore_active': False, 'id': '01ac1df7-76c9-4177-a629-51de75850ee5', 'size': None, 'state': 'active', 'api_response_formats': [], 'hash': '', 'description': 'South Dublin Libraries', 'format': 'CSV', 'mimetype_inner': No

### Selected resource/data ids are:

<br>
<li>South Dublin Libraries - id:'cd8ef111-10da-4abd-a9dc-0e82c9fd256e' format: 'csv'</li>
<li>Limerick Libraries - id:'042801f0-beb9-4d46-83bf-80a82b23a963' format: 'json'</li>
<li>Wexford Libraries - id:'708a6a3c-337a-4109-831a-bb71e3e2a35d' format: 'csv'</li>

Note: id - is the package_id and resc_arr_ind - array index of the resources that need to be extracted

### Function to extract data for csv or json:

In [21]:
def extract_pkg_data(url, pkg_id, resc_arr_ind):
    
    try:
        
        param = {'id': pkg_id}
        response = requests.get(url+'api/3/action/package_show', param)
        results = response.json()
        
        if not results['success']:
            raise SystemError

        if len(results['result']) == 0:
            return ("No package found")
        
        if results['result']['resources'][resc_arr_ind]['format'].lower() == 'csv':
        
            dataset = pd.read_csv(results['result']['resources'][resc_arr_ind]['url'], encoding = 'ISO-8859-1')
            #df = dataset.write('dataframe')

            return dataset
        
        elif ((results['result']['resources'][resc_arr_ind]['format'].lower() == 'geojson') or 
        (results['result']['resources'][resc_arr_ind]['format'].lower() == 'json')):
            
            response_json = requests.get(results['result']['url'])
            data_json = json.dumps(response_json.json())
            
                
            return json.loads(data_json)
        
        else:
            
            return results['result']
        
    
    except SystemError:
        
        print("Request Failure, please check the URL or the parameters")
        sys.exit(1)
        
    except Exception as e:
        
        print(e)
        sys.exit(1)

### Libraries in South Dublin County:

In [24]:
sd_cnty = extract_pkg_data('https://data.gov.ie/', 'cd8ef111-10da-4abd-a9dc-0e82c9fd256e', 0)
sd_cnty.head()

,Council_ID,Administrative_Authority,Name,Address1,Address2,Town,Postcode,County,Phone,Email,Website,Image,Opening_Hours_Monday,Opening_Hours_Tuesday,Opening_Hours_Wednesday,Opening_Hours_Thursday,Opening_Hours_Friday,Opening_Hours_Saturday,WGS84_Latitude,WGS84_Longitude
0,SD1,South Dublin County Council,County Library,Library Square,Belgard Square North,Tallaght,24.0,Dublin,+353 1 462 0073,talib@sdublincoco.ie,http://www.southdublinlibraries.ie/find-librar...,http://www.southdublinlibraries.ie/sites/all/l...,09:45-20:00,09:45-20:00,09:45-20:00,09:45-20:00,09:45-16:30,09:45-16:30,53.288466,-6.373348
1,SD2,South Dublin County Council,Ballyroan Library,Orchardstown Avenue,NaN,Rathfarnham,14.0,Dublin,+353 1 494 1900,ballyroan@sdublincoco.ie,http://www.southdublinlibraries.ie/find-librar...,http://www.southdublinlibraries.ie/sites/all/l...,09:45-20:00,09:45-20:00,09:45-20:00,09:45-20:00,09:45-16:30,09:45-16:30,53.290678,-6.299322
2,SD3,South Dublin County Council,Castletymon Library,Tymon Road North,NaN,Tallaght,24.0,Dublin,+353 1 452 4888,castletymon@sdublincoco.ie,http://www.southdublinlibraries.ie/find-librar...,http://www.southdublinlibraries.ie/sites/all/l...,09:45-17:00,09:45-17:00,09:45-17:00,09:45-17:00,09:45-16:30,09:45-16:30,53.295663,-6.342031
3,SD4,South Dublin County Council,Clondalkin Library,Monastery Road,NaN,Clondalkin,22.0,Dublin,+353 1 459 3315,clondalkin@sdublincoco.ie,http://www.southdublinlibraries.ie/find-librar...,http://www.southdublinlibraries.ie/sites/all/l...,09:45-20:00,09:45-20:00,09:45-20:00,09:45-20:00,09:45-16:30,09:45-16:30,53.320720,-6.388526
4,SD5,South Dublin County Council,Lucan Library,Superquinn Shopping Centre,Newcastle Road,Lucan,NaN,Dublin,+353 1 621 6422,lucan@sdublincoco.ie,http://www.southdublinlibraries.ie/find-librar...,http://www.southdublinlibraries.ie/sites/all/l...,09:45-20:00,09:45-20:00,09:45-20:00,09:45-20:00,09:45-16:30,09:45-16:30,53.347955,-6.453018


#### Data processing:

In [25]:
def southDublin_data_process(dt):
    
    sd_cnty = dt
    sd_cnty.drop(['Council_ID', 'Image', 'Postcode'], axis=1, inplace=True)
    sd_cnty.rename(columns={'WGS84_Latitude': 'Latitude', 'WGS84_Longitude': 'Longitude'}, inplace=True)

    
    try:
        
        for index, cont in sd_cnty.iterrows():

            if (cont['Address2'] is np.nan) and (cont['Town'] is not np.nan):

                sd_cnty.loc[index, 'Addr'] = cont['Town']

            elif (cont['Address2'] is not np.nan) and (cont['Town'] is np.nan):

                sd_cnty.loc[index, 'Addr'] = cont['Address2']

            elif (cont['Address2'] is np.nan) and (cont['Town'] is np.nan):

                sd_cnty.loc[index, 'Addr'] = None

            else:

                sd_cnty.loc[index, 'Addr'] = str(cont['Address2']) +", "+ str(cont['Town'])

        sd_cnty.drop(['Address2', 'Town'], axis=1, inplace=True)
        sd_cnty.rename(columns={'Addr': 'Address2'}, inplace=True)
        #sd_cnty.replace(np.nan, None, inplace=True)
        
    except Exception as e:
        
        print(e)
        sys.exit(1)

    return sd_cnty

sd_cnty = southDublin_data_process(sd_cnty)
sd_cnty.head()

,Administrative_Authority,Name,Address1,County,Phone,Email,Website,Opening_Hours_Monday,Opening_Hours_Tuesday,Opening_Hours_Wednesday,Opening_Hours_Thursday,Opening_Hours_Friday,Opening_Hours_Saturday,Latitude,Longitude,Address2
0,South Dublin County Council,County Library,Library Square,Dublin,+353 1 462 0073,talib@sdublincoco.ie,http://www.southdublinlibraries.ie/find-librar...,09:45-20:00,09:45-20:00,09:45-20:00,09:45-20:00,09:45-16:30,09:45-16:30,53.288466,-6.373348,"Belgard Square North, Tallaght"
1,South Dublin County Council,Ballyroan Library,Orchardstown Avenue,Dublin,+353 1 494 1900,ballyroan@sdublincoco.ie,http://www.southdublinlibraries.ie/find-librar...,09:45-20:00,09:45-20:00,09:45-20:00,09:45-20:00,09:45-16:30,09:45-16:30,53.290678,-6.299322,Rathfarnham
2,South Dublin County Council,Castletymon Library,Tymon Road North,Dublin,+353 1 452 4888,castletymon@sdublincoco.ie,http://www.southdublinlibraries.ie/find-librar...,09:45-17:00,09:45-17:00,09:45-17:00,09:45-17:00,09:45-16:30,09:45-16:30,53.295663,-6.342031,Tallaght
3,South Dublin County Council,Clondalkin Library,Monastery Road,Dublin,+353 1 459 3315,clondalkin@sdublincoco.ie,http://www.southdublinlibraries.ie/find-librar...,09:45-20:00,09:45-20:00,09:45-20:00,09:45-20:00,09:45-16:30,09:45-16:30,53.320720,-6.388526,Clondalkin
4,South Dublin County Council,Lucan Library,Superquinn Shopping Centre,Dublin,+353 1 621 6422,lucan@sdublincoco.ie,http://www.southdublinlibraries.ie/find-librar...,09:45-20:00,09:45-20:00,09:45-20:00,09:45-20:00,09:45-16:30,09:45-16:30,53.347955,-6.453018,"Newcastle Road, Lucan"


### Libraries in Limerick City and County:

Data process/ enrichment for limerick libraries is done in DGI Data Enrichment notebook. Data obtained from DGI Data Enrichment notebook is utilised here.

In [ ]:
lmk_data = pd.read_csv('https://github.com/derilinx/Developer_Corner_DGI/blob/master/lmk_lib_cnt.csv')
lmk_data.head()

### Libraries in Wexford:

In [ ]:
wexford_cnty = extract_pkg_data('https://data.gov.ie/', '517abd28-7460-4512-a8e6-8e66326ed268', 1)
wexford_cnty.head()

##### Data process

In [ ]:
def wexford_data_process(dt):
    
    wexford_cnty = dt
    wexford_cnty.rename(columns = {'WGS84Latitude': 'Longitude', 'WGS84Longitude': 'Latitude'}, inplace = True)
    wexford_cnty = wexford_cnty.loc[:, 'Administrative_Authority':]
    wexford_cnty.drop(['Image', 'OBJECTID_1', 'Postcode'], axis=1, inplace=True)
    wexford_cnty['Addr'] = ''
    
    try:
        
        for index, cont in wexford_cnty.iterrows():

            if (cont['Address2'] is np.nan) and (cont['Town'] is not np.nan):

                wexford_cnty.loc[index, 'Addr'] = cont['Town']

            elif (cont['Address2'] is not np.nan) and (cont['Town'] is np.nan):

                wexford_cnty.loc[index, 'Addr'] = cont['Address2']

            elif (cont['Address2'] is np.nan) and (cont['Town'] is np.nan):

                wexford_cnty.loc[index, 'Addr'] = None

            else:

                wexford_cnty.loc[index, 'Addr'] = str(cont['Address2']) +", "+ str(cont['Town'])

        wexford_cnty.drop(['Address2', 'Town'], axis=1, inplace=True)
        wexford_cnty.rename(columns={'Addr': 'Address2'}, inplace=True)
        wexford_cnty.replace(np.nan, None, inplace=True)
        
    except Exception as e:
        
        print(e)
        sys.exit(1)

    return wexford_cnty

wexford_cnty = wexford_data_process(wexford_cnty)
wexford_cnty.head()

## Final processing of all extracted datasets for consistency:

In [ ]:
def process_all_libraries(sd_cnty, lmk_data, wexford_cnty):
    
    sd = sd_cnty.copy()
    lmk = lmk_data.copy()
    wex = wexford_cnty.copy()
    
    try:
    
        lmk.drop(['Closed', 'sunday'], axis=1, inplace=True)
        lmk.rename(columns={'monday': 'Opening_Hours_Monday', 'tuesday' :'Opening_Hours_Tuesday',
           'wednesday' :'Opening_Hours_Wednesday', 'thursday': 'Opening_Hours_Thursday',
        'friday' :'Opening_Hours_Friday', 'saturday':'Opening_Hours_Saturday', 
                            'Administrative Autority': 'Administrative_Authority'}, inplace=True)

        df = lmk.append([sd, wex], sort=True)
        
        df.replace(np.nan, "NA", inplace=True)
        df.fillna("NA", inplace=True)
        
        
        df['Opening_Hours_Saturday'] = df['Opening_Hours_Saturday'].apply(lambda x: x.strip().replace('\r', ' ').replace('\n', ''))
        df['County'] = df['County'].apply(lambda x: x.replace('County ', ''))
        
        return df
    
    except Exception as e:
        
        print(e)
        sys.exit(1)
    
all_lib_data = process_all_libraries(sd_cnty, lmk_data, wexford_cnty)
all_lib_data.head()

## Visualisation of Libraries location in South Dublin, Limerick and Wexford:

In [ ]:
def location__lib_viz(all_lib_data):

    df= all_lib_data

    map=folium.Map(location=[df['Latitude'].mean(),df['Longitude'].mean()],zoom_start=7,tiles='openstreetmap')


    fg=folium.FeatureGroup(name="Library Locations")

    for lat,lon,name, address1, address2, county, friday, monday, saturday, thursday, tuesday, wednesday, phone, website in zip(df['Latitude'],df['Longitude'],df['Name'], df['Address1'], df['Address2'], df['County'], 
                           df['Opening_Hours_Friday'], df['Opening_Hours_Monday'], df['Opening_Hours_Saturday'], 
                            df['Opening_Hours_Thursday'], df['Opening_Hours_Tuesday'], df['Opening_Hours_Wednesday'],
                           df['Phone'], df['Website']):

        fg.add_child(folium.Marker(location=[lat,lon],
                                   popup=(folium.Popup('<style>table, th, td {border: 1px solid black; border-collapse: collapse;} th, td {padding: 5px; text-align: left;}</style><div><p><b>Name:</b><i> ' +str(name)+ '</i><br><b>County:</b><i> ' + str(county) + '</i><br><b>Address:</b><i> ' + str(address1) + ', ' + str(address2) + '</i><br><b>Phone:</b><i> ' + str(phone) + '</i><br><b>Website: </b> <a href=' + str(website) +'>'+str(website)+'</a><br><br><b>Opening Hours:</b> <br><table style="width:80%"><tr><th>Day</th><th>Hours</th></tr><tr><td>Monday</td><td>' + str(monday) + '</td></tr><tr><td>Tuesday</td><td>' + str(tuesday) + '</td></tr><tr><td>Wednesday</td><td>' + str(wednesday) + '</td></tr><tr><td>Thursday</td><td>' + str(thursday) + '</td></tr><tr><td>Friday</td><td>' + str(friday) + '</td></tr><tr><td>Saturday</td><td>' + str(saturday)+ '</td></tr><tr><td>Sunday</td><td>Closed</td></tr></table></p><p>NA: Not Available</div>')), 
                                   icon=folium.Icon(color='blue',icon_color='orange')))

    map.add_child(fg)
    map.add_child(folium.LayerControl())
    
    return map

location__lib_viz(all_lib_data)